In [20]:
import os
import pandas as pd
from functools import reduce
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.decomposition import NMF

In [21]:
df_list = []
for filename in os.listdir('./data/ml-latest-small'):
    if filename.endswith(".csv"):
        df = pd.read_csv('./data/ml-latest-small/' + f'{filename}')
        df_list.append(df)
        
for df in df_list:
    print(len(df))
    print(df.head(1))

# 9742 films in general
# 100836 ratings
#df_final = reduce(lambda left,right: pd.merge(left,right,on='movieId'), df_list)

9742
   movieId  imdbId  tmdbId
0        1  114709   862.0
3683
   userId  movieId    tag   timestamp
0       2    60756  funny  1445714994
100836
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
9742
   movieId             title                                       genres
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy


In [22]:
# lookup for movie names
df_movie_names = df_list[3]
df_movie_names.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [26]:
# #### deal with small diffs in the names!
# choices = df_movie_names['title'].unique()
# for title in choices:
#     possibilities = process.extract(title, choices, limit=2, scorer=fuzz.token_sort_ratio)
#     #print(possibilities)
#     #print([possible for possible in possibilities if possible[1] > 95])
#     #print([possible for possible in possibilities if possible[1] > 95 and possible[1] < 100])
#     for possible in possibilities:
#         if possible[1] > 95 and possible[1] < 100:
#             print(possible)

In [ ]:
"""
('Miracle on 34th Street (1947)', 96)
('Three Musketeers, The (1973)', 96)
('Hunchback of Notre Dame, The (1939)', 97)
('Nutty Professor, The (1963)', 96)
('Nutty Professor, The (1996)', 96)
('Miracle on 34th Street (1994)', 96)
("Gulliver's Travels (1996)", 96)
('Thief of Bagdad, The (1924)', 96)
('Hunchback of Notre Dame, The (1996)', 97)
('Dr. Jekyll and Mr. Hyde (1931)', 96)
('Thief of Bagdad, The (1940)', 96)
("King Solomon's Mines (1950)", 96)
('Girl Next Door, The (2007)', 96)
('Human Condition II, The (Ningen no joken II) (1959)', 98)
('Dr. Jekyll and Mr. Hyde (1941)', 96)
('Hunchback of Notre Dame, The (1939)', 97)
("King Solomon's Mines (1985)", 96)
('Human Condition I, The (Ningen no joken I) (1959)', 98)
('Blood: The Last Vampire (2009)', 96)
('Employee of the Month (2006)', 96)
('Three Musketeers, The (1993)', 96)
('Ultimate Avengers 2 (2006)', 96)
('Employee of the Month (2004)', 96)
('Death at a Funeral (2010)', 96)
('Girl Next Door, The (2004)', 96)
('Blood: The Last Vampire (2000)', 96)
('Death at a Funeral (2007)', 96)
('Harry Potter and the Deathly Hallows: Part 2 (2011)', 96)
("Mesrine: Public Enemy #1 (L'ennemi public n°1) (2008)", 97)
('Harry Potter and the Deathly Hallows: Part 1 (2010)', 96)
('Nymphomaniac: Volume II (2013)', 98)
('Nymphomaniac: Volume I (2013)', 98)
('Ultimate Avengers (2006)', 96)
("Gulliver's Travels (1939)", 96)

"""

In [11]:
df_ratings = df_list[2]
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
#df.duplicated(subset=['brand'])

In [12]:
df_ratings['userId'].nunique(), df_ratings['movieId'].nunique(), df_movie_names['movieId'].nunique() 
# there are more movie names than rated movies !!!!

(610, 9724, 9742)

In [17]:
### LEFT join movie_names on the movieIds of the rating table
# (to drop the movieIds that do not have a rating - right join would have included them)
df = pd.merge(df_ratings, df_movie_names, on='movieId', how='left')
#df.head(5)

In [18]:
df.shape, df['movieId'].nunique(), df['title'].nunique()

((100836, 6), 9724, 9719)

In [19]:
title_dict = dict(zip(df.title, df.movieId))
len(title_dict)

9719

In [ ]:
a_dictionary = {"a": 1, "b": 2}

reversed_dictionary = {value : key for (key, value) in a_dictionary.items()}

Perform dictionary comprehension


print(reversed_dictionary

### Create a matrix with rows=users, columns=movies, values in the matrix= user_rating a for movie_id=1 etc

In [148]:
# creating ratings matrix (R)
R = df.pivot_table(index='userId', columns='movieId', values='rating', dropna=False)
R.shape, # 610 users , 9724 movieIds

((610, 9724),)

In [149]:
R.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Handle missing data


In [150]:
med_values = R.median().median()

In [151]:
R.fillna(med_values,inplace=True)
R.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,3.5,4.0,3.5,3.5,4.0,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
2,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
3,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
4,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
5,4.0,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5


### Train NMF

In [152]:
m = NMF(n_components=60)

In [153]:
m.fit(R)

/Users/lara/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


NMF(n_components=60)

### Check out the sub-matrices, and the reconstruction error

In [154]:
Q = m.components_
P = m.transform(R)
error = m.reconstruction_err_ #this is an absolute score, so no intuition from looking at in isolation! 
P.shape, Q.shape, error

/Users/lara/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


((610, 60), (60, 9724), 228.46748544822856)

In [156]:
R.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,3.5,4.0,3.5,3.5,4.0,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
2,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
3,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
4,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
5,4.0,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5


In [157]:
new_R = np.dot(P,Q)
pd.DataFrame(new_R.round(1), columns=R.columns, index=R.index)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,3.7,3.3,3.7,3.6,3.6,3.9,3.5,3.5,3.6,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
2,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
3,3.5,3.5,3.4,3.5,3.5,3.4,3.5,3.5,3.5,3.6,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
4,3.0,3.5,3.5,3.5,3.5,3.6,3.5,3.4,3.5,3.4,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
5,3.7,3.5,3.5,3.5,3.5,3.6,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.7,3.3,3.7,3.5,3.3,3.2,2.6,3.5,3.4,3.8,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
607,3.8,3.7,3.7,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
608,2.5,2.1,1.9,3.5,3.5,3.7,3.5,3.5,3.5,4.0,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5


### Make a prediction based on new user input

In [158]:
m

NMF(n_components=60)

Use the NMF model to produce recommendations for one user.

- To map movie names to movie ID’s, construct a {name: id} dictionary
- To deal with small differences in the names, the fuzzywuzzy package is quite useful
- Create an vector of three movies the user likes. Set these to 5 and all others to zero.



In [170]:
# create a dictionary for name: id 
title_dict = dict(zip(df.title, df.movieId))
len(title_dict)

9719

In [169]:
vals = ['Kicking and Screaming (1995)', 'White Squall (1996)', 'In the Bleak Midwinter (1995)']

In [161]:
keys = [title_dict.get(val) for val in vals]
keys

[72, 86, 96]

In [162]:
d_new_user = dict.fromkeys(movie_names_df.movieId,0)

In [163]:
# create ratings_list
ratings_5 = [5 for y in keys]
# create small dict
d_fill = dict(zip(keys, ratings_5))
d_fill

{72: 5, 86: 5, 96: 5}

In [164]:
# update the values with the rated movie ids 
d_new_user.update(d_fill)

In [165]:
#user_P = list(d_new_user.values())
user_P = np.array(list(d_new_user.values()))
user_P.shape 

(9742,)

In [166]:
#new user R - reconstruct R but for this new user only

# column of first matrix = row of second matrix
user_R = np.dot(user_P,Q.T)

ValueError: shapes (9742,) and (9724,60) not aligned: 9742 (dim 0) != 9724 (dim 0)